In [ ]:
import numpy as np

In [ ]:
from jobflow.managers.local import run_locally

In [ ]:
from python_workflow_definition.jobflow import load_workflow_json

In [ ]:
from pyiron_base import Project, job

In [ ]:
from python_workflow_definition.pyiron_base import write_workflow_json

In [ ]:
from python_workflow_definition.jobflow import load_workflow_json

In [ ]:
from quantum_espresso_workflow import (
    calculate_qe as _calculate_qe, 
    generate_structures as _generate_structures, 
    get_bulk_structure as _get_bulk_structure, 
    plot_energy_volume_curve as _plot_energy_volume_curve,
)

In [ ]:
calculate_qe = job(_calculate_qe, output_key_lst=["energy", "volume", "structure"])
generate_structures = job(_generate_structures)
plot_energy_volume_curve = job(_plot_energy_volume_curve)
get_bulk_structure = job(_get_bulk_structure)

In [ ]:
pseudopotentials = {"Al": "Al.pbe-n-kjpaw_psl.1.0.0.UPF"}

In [ ]:
pr = Project("test")
pr.remove_jobs(recursive=True, silently=True)

In [ ]:
structure = get_bulk_structure(
    name="Al",
    a=4.05,
    cubic=True,
    pyiron_project=pr,
)

In [ ]:
calc_mini = calculate_qe(
    working_directory="mini",
    input_dict={
        "structure": structure,
        "pseudopotentials": pseudopotentials,
        "kpts": (3, 3, 3),
        "calculation": "vc-relax",
        "smearing": 0.02,
    },
    pyiron_project=pr,
)

In [ ]:
number_of_strains = 5
structure_lst = generate_structures(  # the generate_structures() function is not available in the workflow graph
    structure=calc_mini.output.structure,
    strain_lst=np.linspace(0.9, 1.1, number_of_strains),
    pyiron_project=pr,
    list_length=number_of_strains,
)

In [ ]:
job_strain_lst = []
for i, structure_strain in enumerate(structure_lst):
    calc_strain = calculate_qe(
        working_directory="strain_" + str(i),
        input_dict={
            "structure": structure_strain,
            "pseudopotentials": pseudopotentials,
            "kpts": (3, 3, 3),
            "calculation": "scf",
            "smearing": 0.02,
        },
        pyiron_project=pr,
    )
    job_strain_lst.append(calc_strain)

In [ ]:
plot = plot_energy_volume_curve(
    volume_lst=[job.output.volume for job in job_strain_lst],
    energy_lst=[job.output.energy for job in job_strain_lst],
    pyiron_project=pr,
)

In [ ]:
write_workflow_json(delayed_object=plot, file_name="workflow_pyiron.json")

In [ ]:
!cat workflow_pyiron.json

In [ ]:
flow = load_workflow_json(file_name="workflow_pyiron.json")

In [ ]:
result = run_locally(flow)
result